In [1]:
'''
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)
'''

Mounted at /content/gdrive/


In [6]:
import pandas as pd
import nltk
import gensim
from gensim import downloader
from nltk.tokenize import RegexpTokenizer
from scipy.spatial import distance
import re
import numpy as np

glove_vectors = gensim.downloader.load('word2vec-google-news-300')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
import torch
from transformers import BertTokenizer, BertModel
import logging
import pandas as pd
import matplotlib.pyplot as plt
from textwrap import wrap
import numpy as np
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from collections import Counter
from functools import reduce

 
num_topics = 50
model_bert = SentenceTransformer('bert-base-nli-max-tokens')               

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/397 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [11]:
df_books = pd.read_csv("./raw_data/Books.csv")
#df_books = pd.read_csv("/content/gdrive/MyDrive/Group-1-Project/data and algorithm/Books.csv")
df_summary = pd.read_csv('booksummaries.txt', sep = '\t')
#df_summary = pd.read_csv('/content/gdrive/MyDrive/Group-1-Project/data and algorithm/booksummaries.txt', sep = '\t')
df_summary.columns = ['Code1', 'Code2', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Code3', 'Book-Summary']

In [13]:
def Compute_ISBN_to_Title(df_merged):
    ISBN_to_Title = {}
    for row in df_merged.index:
        title = df_merged["Book-Title"][row]
        isbn = df_merged["ISBN"][row]
        ISBN_to_Title[isbn] = title
    return ISBN_to_Title

def merge_book_and_summary(df_summary, df_books):
    df_merged = pd.merge(df_summary, df_books, how='inner', on =['Book-Title'])
    df_merged = df_merged.drop(['Code1','Code2','Code3','Publisher', 'Year-Of-Publication_x', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'Book-Author_y', 'Year-Of-Publication_y'], axis=1)
    df_merged = df_merged[df_merged['Book-Title'] != 'Deathstalker Rebellion']
    return df_merged

df_merged = merge_book_and_summary(df_summary, df_books).drop_duplicates(subset=['Book-Title'])
#df_merged.to_csv('Books_Summary_merged.csv')
df_merged

,Book-Title,Book-Author_x,Book-Summary,ISBN
0,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...",345316413X
5,The Plague,Albert Camus,The text of The Plague is divided into five p...,0075536498
7,An Enquiry Concerning Human Understanding,David Hume,The argument of the Enquiry proceeds by a ser...,0872202291
8,All Quiet on the Western Front,Erich Maria Remarque,"The book tells the story of Paul Bäumer, a Ge...",0449213943
12,A Wizard of Earthsea,Ursula K. Le Guin,"Ged is a young boy on Gont, one of the larger...",0689317204
...,...,...,...,...
13776,Gates of Paradise,V. C. Andrews,"A novel about Annie Stonewall, the daughter o...",0671670646
13778,Shelter,Harlan Coben,"After Mickey Bolitar moves in with his uncle,...",0385313896
13781,Remote Control,Andy McNab,The series follows the character of Nick Ston...,0345428056
13785,The Simpsons: A Complete Guide to Our Favorite...,Matt Groening,"{| class=""wikitable"" |- !Seasons covered !Boo...",0060952520


In [14]:
def all_books_to_vecs(books):
  embedding_bert = np.array(model_bert.encode(books))
  return embedding_bert

def reduce_tsne(embedding):
    tsne = TSNE(n_components=2)
    reduced = tsne.fit_transform( embedding )
    
    return reduced

def predict_topics_with_kmeans(embeddings,num_topics):
  kmeans_model = KMeans(num_topics)
  kmeans_model.fit(embeddings)
  topics_labels = kmeans_model.predict(embeddings)
  return topics_labels

df_summary_bert = df_merged
all_summary_list = df_summary_bert["Book-Summary"].tolist()
all_summary_embeddings = all_books_to_vecs(all_summary_list)
df_summary_bert["vector"] = all_summary_embeddings.tolist()
df_summary_bert

,Book-Title,Book-Author_x,Book-Summary,ISBN,vector
0,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...",345316413X,"[-0.296367883682251, 0.8470673561096191, 0.794..."
5,The Plague,Albert Camus,The text of The Plague is divided into five p...,0075536498,"[0.29564276337623596, 1.3197565078735352, 0.35..."
7,An Enquiry Concerning Human Understanding,David Hume,The argument of the Enquiry proceeds by a ser...,0872202291,"[1.2887978553771973, 1.267693281173706, 1.1173..."
8,All Quiet on the Western Front,Erich Maria Remarque,"The book tells the story of Paul Bäumer, a Ge...",0449213943,"[0.2628372311592102, 1.4528498649597168, 1.060..."
12,A Wizard of Earthsea,Ursula K. Le Guin,"Ged is a young boy on Gont, one of the larger...",0689317204,"[-0.12000725418329239, 1.729885220527649, 0.74..."
...,...,...,...,...,...
13776,Gates of Paradise,V. C. Andrews,"A novel about Annie Stonewall, the daughter o...",0671670646,"[0.37652522325515747, 1.5239259004592896, 0.91..."
13778,Shelter,Harlan Coben,"After Mickey Bolitar moves in with his uncle,...",0385313896,"[-0.22981029748916626, 1.2231945991516113, 1.8..."
13781,Remote Control,Andy McNab,The series follows the character of Nick Ston...,0345428056,"[0.23090682923793793, 0.9125347137451172, 0.59..."
13785,The Simpsons: A Complete Guide to Our Favorite...,Matt Groening,"{| class=""wikitable"" |- !Seasons covered !Boo...",0060952520,"[0.09481050819158554, 1.4162527322769165, 0.93..."


In [15]:
#Visualize
def plot_embeddings(embedding, labels,title):

    labels = np.array( labels )
    distinct_labels =  set( labels )
    
    n = len(embedding)
    counter = Counter(labels)
    for i in range(len( distinct_labels )):
        ratio = (counter[i] / n )* 100
        cluster_label = f"cluster {i}: { round(ratio,2)}"
        x = embedding[:, 0][labels == i]
        y = embedding[:, 1][labels == i]
        plt.plot(x, y, '.', alpha=0.4, label= cluster_label)
    plt.legend(title="Topic",loc = 'upper left', bbox_to_anchor=(1.01,1))
    plt.title(title)

embedding_bert_tsne =  reduce_tsne(all_summary_embeddings)
labels_bert_tsne  = predict_topics_with_kmeans(embedding_bert_tsne,num_topics)
df_summary_bert["group"] = labels_bert_tsne
df_summary_bert
#plot_embeddings(embedding_bert_tsne,labels_bert_tsne,"Bert with T-sne")

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


,Book-Title,Book-Author_x,Book-Summary,ISBN,vector,group
0,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...",345316413X,"[-0.296367883682251, 0.8470673561096191, 0.794...",11
5,The Plague,Albert Camus,The text of The Plague is divided into five p...,0075536498,"[0.29564276337623596, 1.3197565078735352, 0.35...",43
7,An Enquiry Concerning Human Understanding,David Hume,The argument of the Enquiry proceeds by a ser...,0872202291,"[1.2887978553771973, 1.267693281173706, 1.1173...",29
8,All Quiet on the Western Front,Erich Maria Remarque,"The book tells the story of Paul Bäumer, a Ge...",0449213943,"[0.2628372311592102, 1.4528498649597168, 1.060...",40
12,A Wizard of Earthsea,Ursula K. Le Guin,"Ged is a young boy on Gont, one of the larger...",0689317204,"[-0.12000725418329239, 1.729885220527649, 0.74...",37
...,...,...,...,...,...,...
13776,Gates of Paradise,V. C. Andrews,"A novel about Annie Stonewall, the daughter o...",0671670646,"[0.37652522325515747, 1.5239259004592896, 0.91...",45
13778,Shelter,Harlan Coben,"After Mickey Bolitar moves in with his uncle,...",0385313896,"[-0.22981029748916626, 1.2231945991516113, 1.8...",34
13781,Remote Control,Andy McNab,The series follows the character of Nick Ston...,0345428056,"[0.23090682923793793, 0.9125347137451172, 0.59...",3
13785,The Simpsons: A Complete Guide to Our Favorite...,Matt Groening,"{| class=""wikitable"" |- !Seasons covered !Boo...",0060952520,"[0.09481050819158554, 1.4162527322769165, 0.93...",14


In [19]:
def compute_Book2Vec(df_sumamry_full):
    title_dict = {}
    summary_list = []

    for row in df_summary_full.index:
        title = df_summary_full['Book-Title'][row]
        summary = df_summary_full['Book-Summary'][row]
        summary_nonum = re.sub(r'\d+', '', summary) #remove number
        tokenizer = RegexpTokenizer(r'\w+') #remove punctuation
        summary_tokens = tokenizer.tokenize(summary_nonum) 
        filtered_summary_tokens = [w for w in summary_tokens if not w.lower() in stop_words] #remove stop words

        summary_vectors = np.zeros(300)
    
        n = 0
        for summary_token in filtered_summary_tokens:
            try:
                summary_token_vec = glove_vectors[summary_token]
            except:
                summary_token_vec = np.zeros(300)
            summary_vectors += summary_token_vec
            n += 1
        summary_list.append (summary_vectors / n)
    
    return summary_list

df_summary_full = df_summary_bert
summary_list = compute_Book2Vec(df_summary_full)
#df_summary = df_summary.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)
df_summary_full = df_summary_full.rename(columns={"vector": "BERT"})
df_summary_full["word2vec"] = summary_list
df_summary_full

,Book-Title,Book-Author_x,Book-Summary,ISBN,BERT,group,word2vec
0,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...",345316413X,"[-0.296367883682251, 0.8470673561096191, 0.794...",11,"[0.045148936920943204, 0.060113709325094175, -..."
5,The Plague,Albert Camus,The text of The Plague is divided into five p...,0075536498,"[0.29564276337623596, 1.3197565078735352, 0.35...",43,"[0.05818041191726434, 0.0684539834006888, 0.00..."
7,An Enquiry Concerning Human Understanding,David Hume,The argument of the Enquiry proceeds by a ser...,0872202291,"[1.2887978553771973, 1.267693281173706, 1.1173...",29,"[0.06308863513119571, 0.024078914002105073, 0...."
8,All Quiet on the Western Front,Erich Maria Remarque,"The book tells the story of Paul Bäumer, a Ge...",0449213943,"[0.2628372311592102, 1.4528498649597168, 1.060...",40,"[0.08981435439165901, 0.08089733888758695, 0.0..."
12,A Wizard of Earthsea,Ursula K. Le Guin,"Ged is a young boy on Gont, one of the larger...",0689317204,"[-0.12000725418329239, 1.729885220527649, 0.74...",37,"[0.07931866853133492, 0.07550479104553444, -0...."
...,...,...,...,...,...,...,...
13776,Gates of Paradise,V. C. Andrews,"A novel about Annie Stonewall, the daughter o...",0671670646,"[0.37652522325515747, 1.5239259004592896, 0.91...",45,"[0.02671554240774601, -0.012985878802360372, -..."
13778,Shelter,Harlan Coben,"After Mickey Bolitar moves in with his uncle,...",0385313896,"[-0.22981029748916626, 1.2231945991516113, 1.8...",34,"[0.07016823508522728, 0.04757967862215909, -0...."
13781,Remote Control,Andy McNab,The series follows the character of Nick Ston...,0345428056,"[0.23090682923793793, 0.9125347137451172, 0.59...",3,"[0.04248046875, 0.0719638400607639, -0.0039070..."
13785,The Simpsons: A Complete Guide to Our Favorite...,Matt Groening,"{| class=""wikitable"" |- !Seasons covered !Boo...",0060952520,"[0.09481050819158554, 1.4162527322769165, 0.93...",14,"[-0.021460793235085228, -0.04207264293323864, ..."


In [ ]:
df_summary_full.to_csv("book-vec-group.csv")